# 8章 Attention

## 8.1 Attentionの仕組み

- 注意機構(attention mechanism)
       - seq2seqをさらに強力に。seq2seqが抱えていた問題を解決可能
       -　必要な情報だけに「注意」を向けさせることができる

### 8.1.1 seq2seqの問題点

- 入力分の長さにかかわらず、常に同じ長さのベクトルに変換する必要あり
- 必要な情報がベクトルからはみ出す可能性あり
![8_1](fig/8_1.png)


### 8.1.2 Encoderの改良

- Encoderの出力
    - 入力される文書の長さに応じて長さが変わるべき
![8_2](fig/8_2.png)
- 各時刻(各単語)の隠れ状態ベクトルを全て利用すれば、入力された単語列と同じ数のベクトルを得ることができる。「ひとつの固定長ベクトル」という制約から解放される

![8_3](fig/8_3.png)
- Encoderが出力するhsという行列は、上図のように、各単語に対応したベクトルの集合とみなすことができる
<br><br>
◆Encoderの改良のまとめ
- Encoderの隠れ状態を全て時刻分取り出し
-　Encoderは入力文の長さに比例した情報をエンコードできるように

### 8.1.3 Decoderの改良①

- Encoder
    - 各単語に対応するLSTMレイヤの隠れ状態ベクトルをhsとして出力。hsがDecoderに渡され、時系列変換が行われる
![8_4](fig/8_4.png)

- 前章のDecoder
    - EncoderのLSTMレイヤにある最後の隠れ状態だけを利用
    - 記号hsを使えば、その最後の行だけ抜き出して、それをDecoderに渡すことになる
    ![8_5](fig/8_5.png)
- Decoderの改良
    - hs全てを活用できるようにする

- Attention
    - 必要な情報にだけ注意を向けさせ、その情報から時系列変換を行う
        - 例：「吾輩は猫である」という文を英語に翻訳するとき、「吾輩=I」「猫=cat」のように、「翻訳先の単語」と対応関係にある「翻訳元の単語」の情報を選び出すこと、そして、その情報を利用して翻訳を行う

    - 全体の枠組み
        - 下図のように「何らかの計算」を行うレイヤを追加する
    ![8_6](fig/8_6.png)
        - 「何らかの計算」
            - 各時刻においてLSTMレイヤの隠れ状態とEncoderからのhsを受け取る
            - 必要な情報だけを選び出し、Affineレイヤへと出力
            - Encoderの最後の隠れ状態ベクトルは、Deocderの最初のLSTMレイヤに渡す<br>
    - 単語のアライメント抽出
         - 各時刻において、Decoderへの入力単語と対応関係にある単語のベクトルをhsから選び出す
             - 例：上図のDecoderが「I」を出力するとき、hsの「吾輩」に対応するベクトルを選び出したい。そのような選び出す操作を「何らかの計算」で実現したい
         - 問題点
             -  選び出すという操作は、微分ができない
                 - ニューラルネットワークの学習は誤差逆伝播法によって行われている為、微分可能な演算を用いなければ、誤差逆伝播方を使うことができない

- 「選ぶ」という操作を微分可能な演算に置き換える方法
    - 「ひとつを選ぶ」のではなく、「すべてを選ぶ」ようにする
    - この時、各単語の重要度(貢献度)を表す「重み」を別途計算するようにする
    ![8_7](fig/8_7.png)
    - 各単語の重要度を表す「重み」(記号aで表す)を利用
        - aは0.0~1.0のスカラで、総和は1になる
        - 各単語の重要度を表す重みaと各単語のベクトルhsから、重み付き和を求めることで目的のベクトルを得る
 ![8_8](fig/8_8.png)
- コンテキストベクトル(c)
    -　現時刻の変換(翻訳)を行うために必要な情報が含まれている
- ここまでの話をコードベースで見たのが下図
    - Encoderが出力するhsと各単語の重みaを適当に作成しその重み付き和を求める
    - 時系列の長さをT=5、隠れ層ベクトルの要素数をH=4として重み付き和を求める過程を示す

In [1]:
import numpy as np
T, H = 5, 4
hs = np.random.randn(T, H)
a = np.array([0.8, 0.1, 0.03, 0.05, 0.02])
ar = a.reshape(5, 1).repeat(4, axis=1)
print(ar.shape)
# (5, 4)
t = hs * ar

print(t.shape)
c = np.sum(t, axis=0)
print(c.shape)

(5, 4)
(5, 4)
(4,)


![8_9](fig/8_9.png)
<br><br>
- NumPyのブロードキャストも利用可
![8_10](fig/8_10.png)

- バッチ処理版の重み付き和の実装は以下の通り

In [2]:
N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
a = np.random.randn(N, T)
ar = a.reshape(N, T, 1).repeat(H, axis=2)
# ar = a.reshape(N, T, 1) # ブロードキャストを使う場合
t = hs * ar
print(t.shape)
# (10, 5, 4)
c = np.sum(t, axis=1)
print(c.shape)

(10, 5, 4)
(10, 4)


- 重み付き和の計算を「計算グラフ」で表したもの

![8_11](fig/8_11.png)

- 逆伝播の実装
    - Repeatの逆伝播はSum
    - Sumの逆伝播はRepeat

```python
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None

    def forward(self, hs, a):
        N, T, H = hs.shape

        ar = a.reshape(N, T, 1)#.repeat(T, axis=1)
        t = hs * ar
        c = np.sum(t, axis=1)

        self.cache = (hs, ar)
        return c

    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2)

        return dhs, da
```

- 上記がコンテキストベクトルを求めるWeight Sumレイヤの実装
- 学習するパラメータは持たないので、self.params=[]とする。

### 8.1.4 Decoderの改良②

- 各単語の重要度を示す重みaの求める方法をみていく
- 下図はDecoderの最初のステップ(時刻)でLSTMレイヤが隠れ状態ベクトルを出力するまでの処理を示したもの
![8_12](fig/8_12.png)
- 上図では、DecoderのLSTMレイヤの隠れ状態ベクトルをhで表している
-　hが、hsの各単語ベクトルとどれだけ似ているかを数値で表すことを目指す
    - ベクトルの内積を利用
        - $ a・b = a_1b_1 + a_2b_2 + ・・・ + a_nb_n$
        - 2つのベクトルがどれだけ同じ方向を向いているかという「類似度」をみる
        - 内積によってベクトル間の類似度を算出するまでの処理は下図
        ![8_13](fig/8_13.png)
        - 類似度の算出結果をsで示す
        - sを正規化するために、Softmax関数を適用する
        ![8_14](fig/8_14.png)
        - Softmax関数を用いることで、その出力であるaの各要素は0.0~0.1で、総和は1となる
        - これまでの処理をコードベースでみたのが下記

```python
import sys
sys.path.append('..')
from common.layers import Softmax
import numpy as np

N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
h = np.random.randn(N, H)
hr = h.reshape(N, 1, H).repeat(T, axis=1)
# hr = h.reshape(N, 1, H) # ブロードキャストの場合

t = hs * hr
print(t.shape)

# (10, 5, 4)
s = np.sum(t, axis=2)
print(s.shape)
# (10, 5)

softmax = Softmax()
a = softmax.forward(s)

print(a.shape)
# (10, 5)
```

- 計算グラフは下記

![8_15](fig/8_15.png)

- 上記計算グラフをAttentionWeightクラスとして実装したのが下図

```python
class AttentionWeight:
    def __init__(self):
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = None

    def forward(self, hs, h):
        N, T, H = hs.shape

        hr = h.reshape(N, 1, H)#.repeat(T, axis=1)
        t = hs * hr
        s = np.sum(t, axis=2)
        a = self.softmax.forward(s)

        self.cache = (hs, hr)
        return a

    def backward(self, da):
        hs, hr = self.cache
        N, T, H = hs.shape

        ds = self.softmax.backward(da)
        dt = ds.reshape(N, T, 1).repeat(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)

        return dhs, dh
```

### 8.1.5 Decoderの改良③

- ここまで、Decoderの改善策を2つのレイヤに分けて説明
    - 8.1.3ではWeight Sumレイヤ
    - 8.1.4ではAttention Weghtレイヤ
- 2つのレイヤを組み合わせる


![8_16](fig/8_16.png)

- Attentionレイヤ
    - Attention Weightレイヤ・・・注意を払い、各単語の重みaを求める
    - Weight Sumレイヤ・・・aとhsの重み付き和を求め、それをコンテキストベクトルcとして出力
    - 計算グラフは下図

![8_17](fig/8_17.png)

- 以上がAttentionレイヤの技術の革新
    - Encoderが渡す情報hsの重要な要素に注意を払い、それを元にコンテキストベクトルを算出し、それを上層へと伝播する
    - 実装は下図

```python
class Attention:
    def __init__(self):
        self.params, self.grads = [], []
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None

    def forward(self, hs, h):
        a = self.attention_weight_layer.forward(hs, h)
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        return out

    def backward(self, dout):
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh
```

- ここでは2つのレイヤ(Weight SumレイヤとAttention Weightレイヤ)による順伝播と逆伝播を行うのみ
- 各単語の重みを後ほど参照できるようにするため、attention_weightというメンバ変数を設定

- Attentionレイヤを、LSTMレイヤとAffineレイヤの間に挿入したのが下図

![8_18](fig/8_18.png)

- 各時刻のAttentionレイヤには、Encoderの出力であるhsが入力される
- LSTMレイヤの隠れ状態ベクトルをAffineレイヤへ入力
- 前章のDecoderに対して、Attentionの情報を追加することになる

![8_19](fig/8_19.png)

- 前章のDecoderに対して、Attentionレイヤによるコンテキストベクトルの情報を追加
- Affineレイヤへは、これまで通りLSTMレイヤの隠れ状態ベクトルを与え、それに追加して、Attentionレイヤのコンテキストベクトルも入力することに

- 最後に、時系列方向に広がった複数のAttentionレイヤをTime Attentionレイヤとしてまとめて実装することにします


![8_20](fig/8_20.png)

- Time Attentionレイヤは、複数のAttentionレイヤをまとめる

```python
class TimeAttention:
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []

        for t in range(T):
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:,t,:])
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)

        return out

    def backward(self, dout):
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(T):
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])
            dhs_enc += dhs
            dhs_dec[:,t,:] = dh

        return dhs_enc, dhs_dec
```

- ここでは、必要な数だけAttentionレイヤを作成し、それぞれが順伝播と逆伝播を行う。
- また、各Attentionレイヤの各単語への重みをattention_weightsとしてリストで持つようにする
- 次節以降では、このAttentionを使って、seq2seqの実装を行なっていく